This is a script to compare the scdesigner AIC/BIC result to the scDesign3 AIC/BIC result.

In [1]:
import anndata
import os
import requests

save_path = "data/example_sce.h5ad"
if not os.path.exists(save_path):
    response = requests.get("https://go.wisc.edu/69435h")
    with open(save_path, "wb") as f:
        f.write(response.content)

example_sce = anndata.read_h5ad(save_path)
example_sce

AnnData object with n_obs × n_vars = 2087 × 100
    obs: 'clusters_coarse', 'clusters', 'S_score', 'G2M_score', 'cell_type', 'sizeFactor', 'pseudotime'
    var: 'highly_variable_genes'
    uns: 'X_name', 'clusters_coarse_colors', 'clusters_colors', 'day_colors', 'neighbors', 'pca'
    obsm: 'PCA', 'UMAP', 'X_pca', 'X_umap'
    layers: 'counts', 'cpm', 'logcounts', 'spliced', 'unspliced'
    obsp: 'connectivities', 'distances'

In [2]:
y = example_sce.X.todense()

Here `y` is the observed value our AIC/BIC will be calculated based on. Below are the R codes to estimate the marginal distributions for scDesign3, running with `rpy2`.

In [3]:
%load_ext rpy2.ipython

Error importing in API mode: ImportError("dlopen(/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/_rinterface_cffi_api.abi3.so, 0x0002): Library not loaded: /Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib\n  Referenced from: <3D361A20-57F1-3C9B-9D2B-6A0F70CF5F35> /Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/_rinterface_cffi_api.abi3.so\n  Reason: tried: '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file), '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file)")
Trying to import in ABI mode.
/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/rpy2/rinterface/__init__.py:1211: UserWarning: Environment variable "PWD" redefined by R and overriding existing variable. Current: "/", R: "/Users/pyl/Desktop/UW Madis

In [4]:
%%R
library(scDesign3)
library(ggplot2)
library(zellkonverter)
theme_set(theme_bw())

example_sce <- readH5AD("data/example_sce.h5ad")


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    ℹ Using stored X_name value 'X'


Registered S3 method overwritten by 'scDesign3':
  method         from  
  predict.gamlss gamlss
Registered S3 method overwritten by 'zellkonverter':
  method                                             from      
  py_to_r.pandas.core.arrays.categorical.Categorical reticulate


In [5]:
%%R
set.seed(123)
PANCREAS_data <- construct_data(
  sce = example_sce,
  assay_use = "counts",
  celltype = "cell_type",
  pseudotime = "pseudotime",
  spatial = NULL,
  other_covariates = NULL,
  corr_by = "1"
)

PANCREAS_marginal <- fit_marginal(
  data = PANCREAS_data,
  predictor = "gene",
  mu_formula = "s(pseudotime, k = 6, bs = 'cr')",
  sigma_formula = "1",
  family_use = "nb",
  n_cores = 2,
  usebam = FALSE
)

Here I will save the marginal distribution coefficients as `param_mean` and `param_sigma`. We should note that the `param_sigma` returned by the scDesign3 marginal model is itself exponentiated, therefore later on when we input it to scdesigner AIC/BIC functions (which expect a logged input), we'll first take the natural log of it.

In [6]:
%%R
mu <- c()
sigma <- c()
for (gene in PANCREAS_marginal) {
  mu <- cbind(mu, gene$fit$coefficients)
  sigma <- c(sigma, gene$fit$family$getTheta(TRUE))
}
write.csv(mu, "data/param_mean.csv")
write.csv(sigma, "data/param_sigma.csv")

write.csv(model.matrix(PANCREAS_marginal$Iapp$fit, what = "mu"), "data/model_matrix.csv") # model matrix is the same for all genes

# Maarginal AIC/BIC

Here we'll compare the AIC/BIC results for scDesign3 marginal estimates. We'll use the `negbin_regression_likelihood` function directly from the negbin estimator file.

In [7]:
from scdesigner.diagnose.aic_bic import marginal
from scdesigner.estimators.negbin import negbin_regression_likelihood

/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_mtx from `annda

In [8]:
import pandas as pd
import numpy as np

mu_coef = pd.read_csv("data/param_mean.csv", index_col=0)
disper_coef = pd.read_csv("data/param_sigma.csv", index_col=0).T
sc3_X = pd.read_csv("data/model_matrix.csv", index_col=0)

Since the marginal AIC/BIC function in scdesigner expects parameter input as a one dimensional Tensor (as a natural follow-up of model training), we'll first convert the parameter coefficients into a one dimensional Tensor.

In [9]:
import torch
n_feature = sc3_X.shape[1]
n_outcomes = y.shape[1]
params = torch.cat((torch.Tensor(mu_coef.values).reshape(1, n_feature*n_outcomes)[0],
           (torch.Tensor(np.log(disper_coef.values))[0])), dim=0)

In [10]:
log_likelihood = torch.sum(negbin_regression_likelihood(params, torch.Tensor(sc3_X.values), torch.Tensor(y)))
marginal(params, log_likelihood, torch.Tensor(y))

(tensor(778925.7500), tensor(782876.1875))

And below are the AIC/BIC calculated directly from scDesign3. Here we notice that the two results vary slightly, and this is due to the fact that scDesign3 uses effective degrees of freedom (edf) instead of number of parameters when calculating AIC/BIC.

In [11]:
%%R
marginals <- lapply(PANCREAS_marginal, function(x){x$fit})
print(paste0("marginal AIC: ", sum(sapply(marginals, stats::AIC))))
print(paste0("marginal BIC: ", sum(sapply(marginals, stats::BIC))))

[1] "marginal AIC: 778898.707033089"
[1] "marginal BIC: 782773.03646551"


# Copula AIC/BIC

Below are codes and functions necessary to run the copula model (since the functions are not public, we have to define them again ourselves).

In [12]:
%%R
sce = example_sce
assay_use = "counts"
marginal_list = PANCREAS_marginal
family_use = "nb"
copula = "gaussian"
n_cores = 2
input_data = PANCREAS_data$dat
important_feature = "all"

if(identical(important_feature, "all")) {
    important_feature <- rep(TRUE, dim(sce)[1])
  }
  
marginals <- lapply(marginal_list, function(x){x$fit})
# find gene whose marginal is fitted
qc_gene_idx <- which(!is.na(marginals))
if(length(family_use) != 1){
  family_use <- family_use[qc_gene_idx]
}
group_index <- unique(input_data$corr_group)
ind <- group_index[1] == "ind"
parallelization = "mcmapply"
BPPARAM = NULL

In [13]:
%%R
convert_n <- function(sce,
                      assay_use,
                      marginal_list,
                      data,
                      DT = TRUE,
                      pseudo_obs = FALSE,
                      epsilon = 1e-6,
                      family_use,
                      n_cores,
                      parallelization,
                      BPPARAM) {
  ## Extract count matrix
  count_mat <-
      t(as.matrix(SummarizedExperiment::assay(sce, assay_use)))
  removed_cell_list <- lapply(marginal_list, function(x){x$removed_cell})
  marginal_list <- lapply(marginal_list, function(x){x$fit})
  # n cell
  ncell <- dim(count_mat)[1]


  mat_function <- function(x, y) {
    fit <- marginal_list[[x]]
    removed_cell <- removed_cell_list[[x]]
    if(length(removed_cell) > 0 && !any(is.na(removed_cell))){
      data<- data[-removed_cell,]
    }
    if (methods::is(fit, "gamlss")) {
      mean_vec <- stats::predict(fit, type = "response", what = "mu", data = data)
      if (y == "poisson" | y == "binomial") {
        theta_vec <- rep(NA, length(mean_vec))
      } else if (y == "gaussian") {
        theta_vec <-
          stats::predict(fit, type = "response", what = "sigma", data = data) # called the theta_vec but actually used as sigma_vec for Gaussian
      } else if (y == "nb") {
        theta_vec <-
          1 / stats::predict(fit, type = "response", what = "sigma", data = data)
        #theta_vec[theta_vec < 1e-3] <- 1e-3
      } else if (y == "zip") {
        theta_vec <- rep(NA, length(mean_vec))
        zero_vec <-
          stats::predict(fit, type = "response", what = "sigma", data = data)
      } else if (y == "zinb") {
        theta_vec <- stats::predict(fit, type = "response", what = "sigma", data = data)
        zero_vec <-
          stats::predict(fit, type = "response", what = "nu", data = data)
      } else {
        stop("Distribution of gamlss must be one of gaussian, poisson, nb, zip or zinb!")
      }
    } else {
      ## if input is from mgcv
      ## Check the family (since sometimes zip and zinb may degenerate into poisson or nb)

      y <- stats::family(fit)$family[1]
      if (grepl("Negative Binomial", y)) {
        y <- "nb"
      }

      mean_vec <- stats::predict(fit, type = "response")
      if (y == "poisson" | y == "binomial") {
        theta_vec <- rep(NA, length(mean_vec))
      } else if (y == "gaussian") {
        theta_vec <- rep(sqrt(fit$sig2), length(mean_vec)) # called the theta_vec but actually used as sigma_vec for Gaussian
      } else if (y == "nb") {
        theta <- fit$family$getTheta(TRUE)
        theta_vec <- rep(theta, length(mean_vec)) #### should it be theta or 1/theta? ####
      } else {
        stop("Distribution of mgcv must be one of gaussian, poisson or nb!")
      }
    }

    ## Mean for Each Cell


    Y <- count_mat[names(mean_vec), x]


    ## Frame
    if (!exists("zero_vec")) {
      zero_vec <- 0
    }
    family_frame <- cbind(Y, mean_vec, theta_vec, zero_vec)
    if (y == "binomial") {
      pvec <- apply(family_frame, 1, function(x) {
        stats::pbinom(x[1], prob = x[2], size = 1)
      })
    } else if (y == "poisson") {
      pvec <- apply(family_frame, 1, function(x) {
        stats::ppois(x[1], lambda = x[2])
      })
    } else if (y == "gaussian") {
      pvec <- apply(family_frame, 1, function(x) {
        gamlss.dist::pNO(x[1], mu = x[2], sigma = abs(x[3]))
      })
    } else if (y == "nb") {
      pvec <- apply(family_frame, 1, function(x) {
        stats::pnbinom(x[1], mu = x[2], size = x[3])
      })
    } else if (y == "zip") {
      pvec <- apply(family_frame, 1, function(x) {
        gamlss.dist::pZIP(x[1], mu = x[2], sigma = abs(x[4]))
      })
    }
    else if (y == "zinb") {
      pvec <- apply(family_frame, 1, function(x) {
        gamlss.dist::pZINBI(x[1],
                            mu = x[2],
                            sigma = abs(x[3]),
                            nu = x[4])
      })
    } else {
      stop("Distribution of gamlss must be one of gaussian, binomial, poisson, nb, zip or zinb!")
    }

    ## CHECK ABOUT THE FIRST PARAM!!!!!
    if (DT) {
      if (y == "poisson") {
        pvec2 <- apply(family_frame, 1, function(x) {
          stats::ppois(x[1] - 1, lambda = x[2]) * as.integer(x[1] > 0)
        })
      } else if (y == "gaussian" | y == "binomial") {
        ## Gaussian is continuous, thus do not need DT.
        ## Binomial seems to be weird to have DT.
        message("Continuous gaussian does not need DT.")
        pvec2 <- pvec
        # pvec2 <- apply(family_frame, 1, function(x){
        # pNO(x[1]-1, mu = x[2], sigma = abs(x[3]))*as.integer(x[1] > 0)
        #})
      } else if (y == "nb") {
        pvec2 <- apply(family_frame, 1, function(x) {
          stats::pnbinom(x[1] - 1, mu = x[2], size = x[3]) * as.integer(x[1] > 0)
        })
      } else if (y == "zip") {
        pvec2 <- apply(family_frame, 1, function(x) {
          ifelse(x[1] > 0, gamlss.dist::pZIP(x[1] - 1, mu = x[2], sigma = abs(x[4])), 0)
        })
      } else if (y == "zinb") {
        pvec2 <- apply(family_frame, 1, function(x) {
          ifelse(x[1] > 0,
                 gamlss.dist::pZINBI(
                   x[1] - 1,
                   mu = x[2],
                   sigma = abs(x[3]),
                   nu = x[4]
                 ),
                 0)
        })
      } else {
        stop("Distribution of gamlss must be one of gaussian, binomial, poisson, nb, zip or zinb!")
      }

      u1 <- pvec
      u2 <- pvec2

      v <- stats::runif(length(mean_vec))
      ## Random mapping
      r <- u1 * v + u2 * (1 - v)
    } else {
      r <- pvec
    }

    if(length(r) < dim(sce)[2]){
      new_r <- rep(1, dim(sce)[2])
      names(new_r) <- colnames(sce)
      new_r[names(r)] <- r
      r <- new_r
    }

    ## Avoid Inf
    idx_adjust <- which(1 - r < epsilon)
    r[idx_adjust] <- r[idx_adjust] - epsilon
    idx_adjust <- which(r < epsilon)
    r[idx_adjust] <- r[idx_adjust] + epsilon

    if (pseudo_obs) {
      r <- rvinecopulib::pseudo_obs(r)
    }
    
    stats::qnorm(
      r,
      mean = 0,
      sd = 1,
      lower.tail = TRUE,
      log.p = FALSE
    )
  }

  paraFunc <- parallel::mcmapply
  if(.Platform$OS.type == "windows"){
    BPPARAM <- BiocParallel::SnowParam()
    parallelization <- "bpmapply"
  }
  if(parallelization == "bpmapply"){
    paraFunc <- BiocParallel::bpmapply
  }
  if(parallelization == "pbmcmapply"){
    paraFunc <- pbmcapply::pbmcmapply
  }
  if(parallelization == "bpmapply"){
    if(class(BPPARAM)[1] != "SerialParam"){
      BPPARAM$workers <- n_cores
    }
    mat <- paraFunc(mat_function, x = seq_len(dim(sce)[1]), y = family_use, SIMPLIFY = TRUE, BPPARAM = BPPARAM)
  }else{
    mat <- paraFunc(mat_function, x = seq_len(dim(sce)[1]), y = family_use, SIMPLIFY = TRUE, mc.cores = n_cores)
  }
  colnames(mat) <- rownames(sce)
  rownames(mat) <- colnames(sce)
  ## Remove inf
  mat[is.infinite(mat)] <- NA

  for (i in 1:ncol(mat)) {
    mat[is.na(mat[, i]), i] <- mean(mat[, i], na.rm = TRUE)
  }
  return(mat)
}

## Calculate the correlation matrix. If use sparse cor estimation, package spcov will be used (it can be VERY SLOW).

cal_cor <- function(norm.mat,
                    important_feature,
                    correlation_function = "default",
                    if_sparse = FALSE,
                    lambda = 0.05,
                    tol = 1e-8,
                    ind = FALSE) {
  if (ind) {
    cor.mat <- diag(rep(1, dim(norm.mat)[2]))
    return(cor.mat)
  }
  else {
    cor.mat <- diag(rep(1, dim(norm.mat)[2]))
    rownames(cor.mat) <- colnames(norm.mat)
    colnames(cor.mat) <- colnames(norm.mat)
    important.mat <- norm.mat[,which(important_feature)]
    if (if_sparse) {
      important_cor.mat <- sparse_cov(important.mat, 
                               method = 'qiu', 
                               operator = 'hard', 
                               corr = TRUE)
    } else {
      important_cor.mat <- correlation(important.mat, correlation_function = correlation_function)
    }

    #s_d <- apply(norm.mat, 2, stats::sd)
    s_d <- matrixStats::colSds(important.mat,na.rm = TRUE)
    if (any(0 == s_d)) {
      important_cor.mat[is.na(important_cor.mat)] <- 0
    }
    cor.mat[rownames(important_cor.mat), colnames(important_cor.mat)] <- important_cor.mat
  }

  n <- dim(cor.mat)[1]

  ### We currently gave up this because the sparse corr calculation is too slow.
  # if (if_sparse) {
  #   ifposd <- matrixcalc::is.positive.definite(cor.mat, tol = tol)
  #   if (!ifposd) {
  #     warning("Cor matrix is not positive defnite! Add tol to the diagnol.")
  #     #diag(cor.mat) <- diag(cor.mat) + tol
  #   }
  #   ## Call spcov
  #   lambda_matrix <- matrix(rep(1, n ^ 2), nrow = n) * lambda
  #   diag(lambda_matrix) <- 0
  #   scor <- spcov::spcov(
  #     cor.mat,
  #     cor.mat,
  #     lambda = lambda_matrix,
  #     step.size  = 100,
  #     trace = 1,
  #     n.inner.steps = 200,
  #     thr.inner = tol
  #   )
  #   cor.mat <- scor$Sigma
  # }
  if (if_sparse) {
  cor.mat <- methods::as(methods::as(methods::as(cor.mat, "dMatrix"), "symmetricMatrix"), "CsparseMatrix")
  }
  cor.mat
}

## Calculate Gaussian copula AIC
cal_aic <- function(norm.mat,
                    cor.mat,
                    ind) {
  if (ind) {
    copula.aic = 0
  } else {
    cor.mat <- as.matrix(cor.mat)
    copula.nop <- (as.integer(sum(cor.mat != 0)) - dim(cor.mat)[1]) / 2

    copula.aic <-
      -2 * (sum(
        mvtnorm::dmvnorm(
          x = norm.mat,
          mean = rep(0, dim(cor.mat)[1]),
          sigma = cor.mat,
          log = TRUE
        )
      ) - sum(rowSums(stats::dnorm(norm.mat, log = TRUE)))) + 2 * copula.nop
  }

  copula.aic
}

## Calculate Gaussian copula BIC
cal_bic <- function(norm.mat,
                    cor.mat,
                    ind) {
  n_obs <- dim(norm.mat)[1]
  if (ind) {
    copula.bic = 0
  } else {
    cor.mat <- as.matrix(cor.mat)
    copula.nop <- (as.integer(sum(cor.mat != 0)) - dim(cor.mat)[1]) / 2

    copula.bic <-
      -2 * (sum(
        mvtnorm::dmvnorm(
          x = norm.mat,
          mean = rep(0, dim(cor.mat)[1]),
          sigma = cor.mat,
          log = TRUE
        )
      ) - sum(rowSums(stats::dnorm(norm.mat, log = TRUE)))) + log(n_obs) * copula.nop
  }

  copula.bic
}

## Similar to the cora function from "Rfast" but uses different functions to calculate column means and row sums.
correlation <- function(x, correlation_function = "default") {
  if(correlation_function == "default") {
    mat <- t(x) - matrixStats::colMeans2(x)
    mat <- mat / sqrt(matrixStats::rowSums2(mat^2))
    tcrossprod(mat)
  } else if (correlation_function == "coop") {
    coop::pcor(x, inplace = TRUE)
  } else {
    stop("correlation_function is either default or coop.")
  }
  
  
}

## Similar to the cova function from "Rfast" but uses different functions to calculate column means and row sums.
covariance <- function(x, correlation_function = "default") {
  if(correlation_function == "default") {
  n <- dim(x)[1]
  m <- sqrt(n) * matrixStats::colMeans2(x)
  s <- (crossprod(x) - tcrossprod(m))/(n - 1)
  s} else if (correlation_function == "coop") {
    coop::covar(x, inplace = TRUE)
  } else {
  stop("correlation_function is either default or coop.")
  }
}

### Sample MVN based on cor
sampleMVN <- function(n,
                      Sigma, 
                      n_cores = n_cores,
                      fastmvn = fastmvn) {
  if_sparse <- methods::is(Sigma, "dsCMatrix")
  p <- dim(Sigma)[1]
  if(if_sparse) {
    CH <- Matrix::Cholesky(methods::as(methods::as(methods::as(Sigma, "dMatrix"), "symmetricMatrix"), "CsparseMatrix"))
    mvnrv <- sparseMVN::rmvn.sparse(n = n, rep(0, p), CH, prec = FALSE)
  } else {
    if(fastmvn) {
      mvnrv <- mvnfast::rmvn(n, mu = rep(0, p), sigma = Sigma, ncores = n_cores)
    } else {
      mvnrv <-
        rmvnorm(n, mean = rep(0, p), sigma = Sigma, checkSymmetry = FALSE, method="eigen")
    }
  }
  
  mvnrvq <- apply(mvnrv, 2, stats::pnorm)

  return(mvnrvq)
}

## fix integer overflow issue when # of gene* # of cell is too larger in rnorm(n * ncol(sigma))
## This function comes from package Mvnorm.
rmvnorm <- function(n, mean = rep(0, nrow(sigma)), sigma = diag(length(mean)),
                    method=c("eigen", "svd", "chol"), pre0.9_9994 = FALSE, checkSymmetry = TRUE)
{
  if (checkSymmetry && !isSymmetric(sigma, tol = sqrt(.Machine$double.eps),
                                    check.attributes = FALSE)) {
    stop("sigma must be a symmetric matrix")
  }
  if (length(mean) != nrow(sigma))
    stop("mean and sigma have non-conforming size")

  method <- match.arg(method)

  R <- if(method == "eigen") {
    ev <- eigen(sigma, symmetric = TRUE)
    if (!all(ev$values >= -sqrt(.Machine$double.eps) * abs(ev$values[1]))){
      warning("Sigma is numerically not positive semidefinite")
    }
    ## ev$vectors %*% diag(sqrt(ev$values), length(ev$values)) %*% t(ev$vectors)
    ## faster for large  nrow(sigma):
    t(ev$vectors %*% (t(ev$vectors) * sqrt(pmax(ev$values, 0))))
  }
  else if(method == "svd"){
    s. <- svd(sigma)
    if (!all(s.$d >= -sqrt(.Machine$double.eps) * abs(s.$d[1]))){
      warning("Sigma is numerically not positive semidefinite")
    }
    t(s.$v %*% (t(s.$u) * sqrt(pmax(s.$d, 0))))
  }
  else if(method == "chol"){
    R <- chol(sigma, pivot = TRUE)
    R[, order(attr(R, "pivot"))]
  }

  retval <- matrix(stats::rnorm(as.double(n) * ncol(sigma)), nrow = n, byrow = !pre0.9_9994) %*%  R
  retval <- sweep(retval, 2, mean, "+")
  colnames(retval) <- names(mean)
  retval
}

In [14]:
%%R
newmat <- convert_n( 
        sce = sce[qc_gene_idx,],
        assay_use = assay_use,
        marginal_list = marginal_list[qc_gene_idx],
        data = input_data,
        DT = TRUE,
        pseudo_obs = FALSE,
        n_cores = n_cores,
        family_use = family_use,
        epsilon = 1e-6,
        parallelization = parallelization,
        BPPARAM = BPPARAM
      )

## select important genes
if(is.vector(important_feature) & methods::is(important_feature,"logical")){
  if(length(important_feature) != dim(sce)[1]){
    stop("The important_feature should either be 'auto' or a logical vector with the length equals to the number of genes in the input data")
  }
}else{
  if(is.numeric(important_feature)){
    stopifnot(important_feature <= 1)
    gene_zero_prop <- apply(as.matrix(SummarizedExperiment::assay(sce, assay_use)), 1, function(y){
      sum(y < 1e-5) / dim(sce)[2]
    })
    important_feature <- gene_zero_prop < important_feature ## default zero proportion in scDesign2 is 0.8.
    names(important_feature) <- rownames(sce)
  }else{
    stop("The important_feature should either be a numeric value or a logical vector with the length equals to the number of genes in the input data")
  }
}

important_feature <- important_feature[qc_gene_idx]


corr_group <- as.data.frame(input_data$corr_group)
colnames(corr_group) <- "corr_group"

In [15]:
%%R
newmvn.list <- lapply(group_index, function(x,
                                   sce,
                                   newmat,
                                   corr_group,
                                   ind,
                                   n_cores,
                                   important_feature) {
        message(paste0("Copula group ", x, " starts"))
        curr_index <- which(corr_group[, 1] == x)
       
        #message(paste0("Group ", group_index, " Start"))
        curr_mat <- newmat[curr_index, , drop = FALSE]
        #message("Cal MVN")
        cor.mat <- cal_cor(
          curr_mat,
          important_feature = important_feature,
          if_sparse = FALSE,
          correlation_function = "default",
          lambda = 0.05,
          tol = 1e-8,
          ind = ind
        )
        #message("Sample MVN")
        #new_mvu <- sampleMVN(n = curr_ncell,
        #                     Sigma = cor.mat)
        #message("MVN Sampling End")
        #rownames(new_mvu) <- curr_ncell_idx
        
        #message("Cal AIC/BIC Start")
        model_aic <- cal_aic(norm.mat = newmat,
                             cor.mat = cor.mat,
                             ind = ind)
        model_bic <- cal_bic(norm.mat = newmat,
                             cor.mat = cor.mat,
                             ind = ind)
        #message("Cal AIC/BIC End")
        return(
          list(
            #new_mvu = new_mvu,
            model_aic = model_aic,
            model_bic = model_bic,
            cor.mat = cor.mat
          )
        )
      }, sce = sce, 
      newmat = newmat, 
      ind = ind, 
      n_cores = n_cores, 
      corr_group = corr_group,
      important_feature = important_feature)
print(paste0("copula AIC: ", newmvn.list[[1]]$model_aic))
print(paste0("copula BIC: ", newmvn.list[[1]]$model_bic))


[1] "copula AIC: -8769.93400064099"
[1] "copula BIC: 19165.3063893912"


Copula group 1 starts


And now I'll compare the scDesign3 copula AIC/BIC result with the AIC/BIC result calculated by scdesigner.

In [16]:
from scdesigner.diagnose.aic_bic import gaussian_copula
from scipy.stats import nbinom, norm

For now I will use the uniformizer as defined in scDesign3. I will incorporate the uniformizer within the aic/bic function later when I figure out which one to use.

In [17]:
def negbin_uniformizer(parameters, x, y, epsilon=1e-3):
    np.random.seed(42)
    r, mu = np.exp(parameters["gamma"]), np.exp(x @ parameters["beta"])
    u1 = nbinom(n=r, p=r / (r + mu)).cdf(y)
    u2 = np.where(y>0, nbinom(n=r, p=r / (r + mu)).cdf(y-1), 0)
    v = np.random.uniform(size=y.shape)
    # may need to add extra step for selecting partial cells?
    p = np.clip(v * u1 + (1 - v) * u2, epsilon, 1 - epsilon)
    return p

Since the copula AIC/BIC is calculated based on the uniformized result and its corresponding covariance, we'll first save the uniformized result (so that the randomness is controlled) and estimate the covariance based on it to obtain the copula AIC/BIC.

In [18]:
u = negbin_uniformizer({'gamma': np.log(disper_coef.values), 'beta': mu_coef.values}, sc3_X, y)
z = norm().ppf(u) # this is equivalent to the newmat variable in the R code, which is the normalized matrix
pd.DataFrame(z).to_csv("data/newmat1.csv", index=True)

In [19]:
%%R
newmat1 <- read.csv("data/newmat1.csv", row.names = 1)
cor.mat <- cal_cor(
          as.matrix(newmat1),
          important_feature = important_feature,
          if_sparse = FALSE,
          correlation_function = "default",
          lambda = 0.05,
          tol = 1e-8,
          ind = ind
        )
write.csv(cor.mat, "data/cor_mat.csv")

In [20]:
covariance = pd.read_csv("data/cor_mat.csv", index_col=0)
gaussian_copula(covariance.values, u)

(-8830.157223613933, 19105.08316641821)

We can see the copula AIC/BIC results are very similar, and if we input the exact normalized matrix and its corresponding covariance into the scDesign3 function, we'll get the exact copula AIC/BIC.

In [21]:
%%R
print(paste0("copula AIC: ", cal_aic(as.matrix(newmat1), cor.mat, ind=FALSE)))
print(paste0("copula BIC: ", cal_bic(as.matrix(newmat1), cor.mat, ind=FALSE)))

[1] "copula AIC: -8830.15722361475"
[1] "copula BIC: 19105.0831664174"


# scdesigner AIC/BIC example

Now we are going to extract AIC/BIC from our scdesigner models. Since we are using a uniformizer different from what scdesigner is currently using, we'll define the simulator manually.

In [22]:
from scdesigner.estimators.negbin import negbin_regression_array, format_negbin_parameters
from scdesigner.estimators import gaussian_copula_factory as gcf
from scdesigner.simulators.glm_simulator import glm_simulator_generator
from scdesigner.samplers import negbin_copula_sample
from scdesigner.predictors import negbin_predict

In [23]:
negbin_copula_array = gcf.gaussian_copula_array_factory(
    negbin_regression_array, negbin_uniformizer
)

negbin_copula = gcf.gaussian_copula_factory(
    negbin_copula_array, format_negbin_parameters
)

NegBinCopulaSimulator = glm_simulator_generator(
    "NegBinCopulaSimulator",
    negbin_copula,
    negbin_copula_sample,
    negbin_predict,
)

In [24]:
formula = "~ bs(pseudotime, degree=5)"
nbcopula = NegBinCopulaSimulator()
nbcopula.fit(example_sce, formula)

/Users/pyl/Desktop/UW Madison/Graduate/2025 Spring/sc project/scDesigner/scdesigner/src/scdesigner/data/group.py:107: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return self.list[idx]
Epoch 1/40:   0%|          | 0/5 [00:00<?, ?it/s]/Users/pyl/Desktop/UW Madison/Graduate/2025 Spring/sc project/scDesigner/scdesigner/src/scdesigner/data/group.py:107: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return self.list[idx]


Marginal and copula AIC and BIC are included as instances within each simulator class.

In [25]:
print("Marginal AIC:", nbcopula.marginal_aic)
print("Marginal AIC:", nbcopula.marginal_bic)
print("Copula BIC:", nbcopula.copula_aic)
print("Copula BIC:", nbcopula.copula_bic)

Marginal AIC: 793850.0
Marginal AIC: 797800.4375
Copula BIC: -37257.83873335656
Copula BIC: -9322.598343324418


Which are the same as what we calculated directly using AIC/BIC methods.

In [26]:
from formulaic import model_matrix
import torch

X = model_matrix(formula, example_sce.obs).values

params = torch.cat((torch.Tensor(nbcopula.params['beta'].values).reshape(1, n_feature*n_outcomes)[0],
           (torch.Tensor(np.log(nbcopula.params['gamma'].values)[0]))), dim=0)
log_likelihood = torch.sum(
    negbin_regression_likelihood(params, torch.Tensor(X), torch.Tensor(y))
)
marginal(params, log_likelihood, torch.Tensor(y))

(tensor(793849.8750), tensor(797800.3125))

In [27]:
covariance = nbcopula.params['covariance'].values
u = negbin_uniformizer({'beta': nbcopula.params['beta'].values,
                        'gamma': nbcopula.params['gamma'].values}, X, y)
gaussian_copula(covariance, u)

(-37257.889846971026, -9322.649456938881)